In [1]:
import pandas as pd
import math
import snappy
import sage

In [2]:
pd.set_option('display.max_colwidth', None)

# Table of Knot Invariants for $T(4,n)$
Knot signature, Alexander polynomial, Arf, upsilon, etc.

### TODO - implement lower bounds from Binns, et al. (https://arxiv.org/pdf/2109.09187)

### Symmetric Alexander Polynomial

In [3]:
def symmetric_alexander_poly(K):
    alex = K.alexander_polynomial(norm=False)
    alex_sym = alex.shift(-min(alex.exponents()) // 2) 
    if (alex_sym(1) == -1):
        alex_sym *= -1
    return alex_sym

In [4]:
T45 = snappy.Link('T(4,5)')
T45.alexander_polynomial()

t^12 - t^11 + t^8 - t^6 + t^4 - t + 1

In [5]:
T45.alexander_polynomial()(1)

1

In [6]:
symmetric_alexander_poly(T45)

t^-6 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^6

In [7]:
symmetric_alexander_poly(T45)(1)

1.0

In [8]:
for q in range(6, 10):
    if math.gcd(5,q) != 1: continue
    
    knot_name = f'T(5,{q})'
    knot = snappy.Link(knot_name)
    print(f'{knot_name}: {symmetric_alexander_poly(knot)}')

T(5,6): t^-10 - t^-9 + t^-5 - t^-3 + 1 - t^3 + t^5 - t^9 + t^10
T(5,7): t^-12 - t^-11 + t^-7 - t^-6 + t^-5 - t^-4 + t^-2 - t^-1 + 1 - t + t^2 - t^4 + t^5 - t^6 + t^7 - t^11 + t^12
T(5,8): t^-14 - t^-13 + t^-9 - t^-8 + t^-6 - t^-5 + t^-4 - t^-3 + t^-1 - 1 + t - t^3 + t^4 - t^5 + t^6 - t^8 + t^9 - t^13 + t^14
T(5,9): t^-16 - t^-15 + t^-11 - t^-10 + t^-7 - t^-5 + t^-2 - 1 + t^2 - t^5 + t^7 - t^10 + t^11 - t^15 + t^16


### $d$-invariant for $T(p,q)$

Proposition 3.8. (Fairchild, et al.)
Let
$$
\Delta_{p,q}(t) = a_0 + \sum_{i=1}^d{a_i(t^i+t^{-i})}
$$
be the symmetric presentation of the Alexander polynomial.
Then $d(T(p,q))$ is given by
$$
d(p,q) = 2t_0 = 2\sum_{i=1}^d{ja_j}.
$$

Note: we could also compute this using Theorem 1.1:
$$
d(p,q) = 2\left(
\left\lfloor\frac{p}{2}\right\rfloor +
\sum_{k=0}^{\lfloor\frac{p}{2}\rfloor-1}{
\left\lfloor \frac{(p-1-2k)q-p-1}{2p} \right\rfloor
}
\right).
$$

In [9]:
def d_invariant(T_pq):
    alex_sym = symmetric_alexander_poly(T_pq)
    coeff_dict = alex_sym.dict()
    pos_exponents = [k for k in coeff_dict.keys() if k>0]
    
    d_T = 0
    for j in pos_exponents:
        d_T += j * coeff_dict[j] # j * a_j
        
    return 2 * d_T

### Make the table for arbitrary $(p,q)$

In [10]:
def make_table(p, max_q=None):
    if max_q is None:
        max_q = p+12
        
    invariants = []
    for q in range(p+1, max_q+1):
        if math.gcd(p,q) != 1: continue
    
        knot_name = f'T({p},{q})'
        knot = snappy.Link(knot_name)
        sage_knot = knot.sage_link()
    
        floer = knot.knot_floer_homology()
        #print(floer)
        
        invariants.append([q, knot_name, knot.signature(), sage_knot.arf_invariant(), floer['nu'], d_invariant(knot)])
        
    return pd.DataFrame(data=invariants, columns=['$q$', 'Knot', '$\\sigma$', 'Arf', '$\\nu$', '$d$'])

In [11]:
df = make_table(4, max_q=15)
df

,$q$,Knot,$\sigma$,Arf,$\nu$,$d$
0,5,"T(4,5)",-8,1,6,6
1,7,"T(4,7)",-14,0,9,8
2,9,"T(4,9)",-16,0,12,8
3,11,"T(4,11)",-22,1,15,10
4,13,"T(4,13)",-24,1,18,14
5,15,"T(4,15)",-30,0,21,16


Observe that Remark 2.2 from Fairchild, et al. holds:

For odd $q$,
$$
\text{Arf}(T(p,q)) =
\begin{cases}
0 & p \text{ odd or } q\equiv\pm1\bmod 8, \\
1 & p \text { even and } q \equiv\pm 3\bmod 8.
\end{cases}
$$

### TODO - fix $d$-invariant implementation

Some data for verifying that `d_invariant` is working properly.

Check these against Proposition 3.8 from Fairchild, et al.

<div>
<center>
<img src="fairchild_prop_3-8.png" width="500"/>
</center>
</div>

In [12]:
make_table(5, max_q=11)

,$q$,Knot,$\sigma$,Arf,$\nu$,$d$
0,6,"T(5,6)",-16,1,10,6
1,7,"T(5,7)",-16,0,12,8
2,8,"T(5,8)",-20,1,14,10
3,9,"T(5,9)",-24,0,16,12
4,11,"T(5,11)",-24,0,20,12


$d(5,q)$ has correct residue class, but shouldn't be negative?

In [ ]:
make_table(6, max_q=17)

$d(6,n)$ looks good!

# Lower bounds on $\gamma_4(T(4,n))$

From Binns, et al., p. 10.
<div>
<center>
    <img src="binns_bounds_T4q.png" width="300"/>
</center>
</div>

Let's see for which $n$ does $\gamma_4(T(4,n))\ge 2$ using Proposition 2.3 from Fairchild, et al.

Proposition 2.3. Given a knot $K$ in $S^3$, if $$\sigma(K)+4\text{Arf}(K)\equiv 4\bmod 8$$ then $\gamma_4(K)\ge2$.

In [ ]:
sig_arf = df['$\\sigma$'] + 4 * df['Arf']
sig_arf_lb_cond = sig_arf % 8
lb_applies = ['✓' if x == 4 else ' ' for x in sig_arf_lb_cond]
pd.DataFrame(data=zip(df['$q$'], sig_arf, sig_arf_lb_cond, lb_applies),
             columns=['$q$', '$\\sigma$/Arf expr', 'expr mod 8', 'Lower bound applies?'])
             #columns=['q', '$\\sigma(K)+4\\text{Arf}(K)$', '$\\sigma(K)+4\\text{Arf}(K)\\bmod8$', 'Lower bound applies?'])

### TODO - look into fixing column header latex formatting

https://stackoverflow.com/questions/72027874/python-formatting-a-pandas-dataframe-head-with-latex

Now let's compute the lower bound in Theorem 2.6, again from Fairchild, et al.

For a knot $K$ in $S^3$,
$$
\left| \nu(K)+\frac{\sigma(K)}{2} \right| \le \gamma_4(K).
$$

In [ ]:
lb_nu = abs(df['$\\nu$'] + 0.5 * df['$\\sigma$'])

In [ ]:
pd.DataFrame(data=zip(df['$q$'], lb_nu),
             columns=['$q$', '$\\nu$ lower bound'])
             #columns=['q', '$\\left| \\nu(K)+\\frac{\\sigma(K)}{2} \\right|$'])

#.style.set_table_styles([dict(selector="th", props=[('max-width', '120px'),
#                                ('text-overflow', 'ellipsis'), ('overflow', 'hidden')])])

## TODO - when can we apply Lemma 2.6 (Binns, et al.)?
it says $K\subseteq S^3$ with $\sigma(K) < 2\nu(K)$ s.t. double branched cover of $S^3$ along $K$ is an integer homology 3-sphere and $\delta(K)<0$ where $\delta(K)$ is the concordance invariant defined by Manolescu and Owens.
Then
$$
\nu(K) - \frac{\sigma(K)}{2} + 1 \le \gamma_4(K).
$$

Lastly, let's compute the lower bound from Theorem 2.7 using the $d$-invariant.

For a knot $K$,
$$
\frac{\sigma(K)}{2} - d(K) \le \gamma_4(K).
$$

In [ ]:
lb_d = 0.5 * df['$\\sigma$'] - df['$d$']

In [ ]:
pd.DataFrame(data=zip(df['$q$'], lb_d),
             columns=['$q$', '$d$ lower bound'])
             #columns=['q', '$\\frac{\\sigma(K)}{2} - d(K) \\le \\gamma_4(K)$'])

# Putting it all together

Computing the maximum lower bound according to the three given in Fairchild, et al.

In [ ]:
# non-orientable 4-genus is always a positive integer (>= 1),
# so just give the trivial lower bound if the signature/arf lb doesn't apply
sig_arf_lb = [2 if x == 4 else 1 for x in sig_arf_lb_cond]

In [ ]:
max_lbs = [max(x) for x in zip(sig_arf_lb, lb_nu, lb_d)]

In [ ]:
pd.DataFrame(zip(df['$q$'], df['Knot'], sig_arf_lb, lb_nu, lb_d, max_lbs),
             columns=['$q$', 'Knot', 'arf+trivial lb', '$\\nu$ lb', '$d$ lb', 'maximum lb'])